<a href="https://colab.research.google.com/github/Akshata4/data_engineering/blob/main/pyspark/PySpark_Working_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark findspark

In [ ]:
!java -version

openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [ ]:
import findspark
findspark.init()

In [ ]:
%cd /content/

/content


In [4]:
from pyspark.sql import SparkSession
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.regression import LinearRegression

In [5]:
spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [6]:
df = spark.createDataFrame([(1, "hello"), (2, "world")], ["id", "word"])

In [7]:
df.show()

+---+-----+
| id| word|
+---+-----+
|  1|hello|
|  2|world|
+---+-----+



In [8]:
spark.stop()